# Data Analysis Extra - Playlists

Como análisis extra pensé que podría analizar las plasyslist que crearon lso usuarios durante la pandemia, podría dar indicadores de qué tipo de música prefiere escuchar durante una pandemia.

el proceso a realizar es buscar las playlist que tengan en el nombre alguna palabra relacionada con la pandemia, me he limitado al español

In [ ]:
# Ver : https://towardsdatascience.com/what-music-do-people-listen-to-during-the-coronavirus-a-report-using-data-science-1a2035d12430
# Leer: https://blog.chartmetric.com/covid-19-effect-on-the-global-music-business-part-1-genre/

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from datetime import datetime

# Meter credenciales de Spotify
passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [31]:
# Vemos como está montada la estructura de información en la API de Spotify, para buscar una palabra en los nombres de las playlists
print(sp.search('q=fernando simón', type='playlist'))

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=q%3Dfernando+sim%C3%B3n&type=playlist&offset=0&limit=10', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.co

In [29]:
# Creamos un script ya completo para buscar más palabras y sacar solo el total de listas en las que hay esa palabra en el título
for word in ["covid","corona","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['total'])

3419
3131
1918
416
2346
907
1


In [ ]:
# Extracción de las playlist con dichos nombres, domcumentación: https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/

# Función para extrar determinados features de la playlist
def batch_proccess(x,lista):
    for i in x['playlists']['items']:
        lista.append({"name" : i['name'], "total":i["tracks"]['total'], "uri":i["uri"], "added_at":i["tracks"]['items']["added_at"]})

# Sacar un listado de las playlist, en bloques de 50, que es el máximo que te permite la API
list_of_playlists = []
for term in ["covid","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']:
    for i in range(0,2000,50):
        try:
            init_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
            batch_proccess(init_data,list_of_playlists)
        except:
            print("Error")

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [35]:
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)
playlists

,name,total,uri
0,Calma,151,spotify:playlist:37i9dQZF1DWY5LGZYBBHHz
1,quarantine Vibes | Covid 19 |,40,spotify:playlist:6zjC569rXoYGOLwPUt37WW
2,COVID-19 Quarantine Party,29,spotify:playlist:1Tn6OrkJhOYO5u6y16erqB
3,Canciones Mas Virales que el Coronavirus 🦠🎶,100,spotify:playlist:4Y2h9TZNZersxSYeXujCNu
4,Drunksouls COVID19 quarantine discovery: music...,49,spotify:playlist:64KUBwKGkUEqIU08HDhJ94
...,...,...,...
6998,My Playlist 3.5,42,spotify:playlist:2FykDxR75paDjs4cYDWAZX
6999,nCov-2019,43,spotify:playlist:3cqPni9PbXLE7WukiRN9Rc
7000,FS Corona-voidance Jams,13,spotify:playlist:4AntDyOlereHExywYCgZ1j
7001,Isolamento com Ashton,11,spotify:playlist:2L41j2odRvSvBZnH4AZcdV


In [ ]:
'''# Loop para sacar features de todas las canciones de una lista
# OJO, tarda bastante dependendo del número de canciones, por lo que antes hago un testeo
import time
list_toextract = test_list

tracks = []

Time1 = datetime.now()


for i in range(len(list_toextract)):
    time.sleep(.5)
    track = getTrackFeatures(list_toextract[i])
    tracks.append(track)
    
Time2 = datetime.now()


Time1_v = datetime.now()

for i in range(len(list_toextract)):
    # time.sleep(.5)
    track = getTrackFeatures(list_toextract[i])
    tracks.append(track)
    
Time2_v = datetime.now()

print("Tiempo 1:", Time2 -Time1)
print("Tiempo 2:", Time2_v -Time1_v)
'''

In [ ]:
for uri in pl_ids:

           results = self.sp.user_playlist_tracks(uri.split(':')[2], uri.split(':')[4])
           tracks = results['items']

           # Loops to ensure I get every track of the playlist
           while results['next']:
               results = self.sp.next(results)
               tracks.extend(results['items'])

       return tracks